# COMP5318 - Machine Learning and Data Mining: Assignment 1

In [1]:
import pandas as pd
import os
print(os.listdir("./Input/train"))
pd.set_option('display.max_columns', 10)

from IPython.display import set_matplotlib_formats, display
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support
import graphviz

# TODO
from sklearn.ensemble import BaggingClassifier

%matplotlib inline

['train.csv']


In [2]:
# train.csv including feature and label using for training model.
data_train_df = pd.read_csv('./Input/train/train.csv') 

In [3]:
# Selecting input feature
data_train_feature = data_train_df.loc[:, "v1":"v784"].to_numpy()

# Selecting output lable 
data_train_label = data_train_df.label.to_numpy()

In [4]:
# Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(
    data_train_feature, data_train_label, random_state=0, stratify=data_train_label, test_size=0.01)

In [5]:
# Performance Metrics Calculator Helper 
def performance(y_true, y_pred, type):
    precision_test = precision_recall_fscore_support(y_true, y_pred, average='macro')
    print("Accuracy on " + type + " set: {:.3f}".format(accuracy_score(y_true, y_pred)))
    print("Precision on " + type + " set: {:.3f}".format(precision_test[0]))
    print("Recall on " + type + " set: {:.3f}".format(precision_test[1]))
    print("F-Score on " + type + " set: {:.3f}".format(precision_test[2]))

## DATA PRE-PROCESSING FOR TRAINING DATA

In [6]:
# Normalisation
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)

pd.DataFrame(X_train_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
29695,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
29696,0.0,0.0,0.016807,0.020548,0.010101,...,0.0,0.0,0.0,0.003922,0.0
29697,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0
29698,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0


In [7]:
# Dimension Reduction
pca = PCA(n_components=0.95).fit(X_train_norm)

X_train_pca = pca.transform(X_train_norm)

pd.DataFrame(X_train_pca)

,0,1,2,3,4,...,182,183,184,185,186
0,-2.366617,5.768976,4.584759,0.942187,-1.642634,...,0.010944,0.051414,-0.110176,-0.030686,0.011074
1,-2.300867,6.339254,4.397728,1.083719,-0.823912,...,-0.207512,0.020794,-0.006619,0.069559,-0.148439
2,-0.735001,4.485583,4.583494,0.896705,-2.913301,...,0.108104,0.005838,-0.033588,-0.227265,-0.243757
3,-6.739240,1.001366,-1.673596,-1.458339,0.821618,...,-0.190961,-0.021111,-0.173409,-0.034948,-0.028491
4,-6.032721,1.304805,-0.561309,-2.131263,0.840133,...,0.062139,-0.086417,-0.233272,-0.021069,0.218451
...,...,...,...,...,...,...,...,...,...,...,...
29695,-3.035287,-2.091845,-3.356072,0.763588,-0.003012,...,0.163366,0.078273,0.035932,-0.075215,0.054444
29696,6.131349,3.504712,-2.403611,0.483445,-1.695134,...,-0.120049,-0.007107,0.009229,-0.130502,0.152417
29697,-7.487262,-2.033362,-2.612905,0.530480,0.163973,...,-0.063564,0.098505,0.016814,0.134421,-0.061286
29698,1.884568,-3.181017,-0.544613,-0.325355,0.104992,...,0.088689,0.017763,0.055164,-0.036278,0.034351


## SVM

In [8]:
%%time
svm = SVC(C=10, gamma=0.01)
svm.fit(X_train_pca, y_train)

y_pred_train = svm.predict(X_train_pca)
performance(y_train, y_pred_train, 'train')

Accuracy on train set: 0.955
Precision on train set: 0.955
Recall on train set: 0.955
F-Score on train set: 0.955
CPU times: user 2min 44s, sys: 732 ms, total: 2min 45s
Wall time: 3min 11s


## SVM BAGGING 

In [9]:
bclf = BaggingClassifier( 
    base_estimator=SVC(C=10, gamma=0.01),
    n_estimators=10, random_state=0
)

bclf.fit(X_train_pca, y_train)
y_pred_train = bclf.predict(X_train_pca)
performance(y_train, y_pred_train, 'train')

Accuracy on train set: 0.951
Precision on train set: 0.952
Recall on train set: 0.951
F-Score on train set: 0.951


## DATA PRE-PROCESSING FOR BLIND TESTING DATA

In [ ]:
# test_input.csv includes 5000 samples used for label prediction. Test samples do not have labels.
data_test_df = pd.read_csv('./Input/test/test_input.csv', index_col=0) 

In [ ]:
# Data Normalisation
output_test_norm = scaler.transform(data_test_df.to_numpy())

pd.DataFrame(output_test_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
2,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
3,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4996,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4997,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4998,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0


In [ ]:
# Dimension Reduction
output_test_pca = pca.transform(output_test_norm)

pd.DataFrame(output_test_pca)

,0,1,2,3,4,...,182,183,184,185,186
0,1.969176,-5.465572,1.963855,-1.911317,-1.070610,...,0.152434,-0.141998,-0.126998,0.048715,0.005861
1,1.910601,-5.687303,1.310701,-1.384259,-0.643309,...,0.004067,-0.018628,-0.005452,0.076242,0.024554
2,0.464712,-5.857919,1.600550,-1.983144,-2.696122,...,-0.003344,-0.097612,-0.147449,-0.040962,0.034372
3,-0.377191,-3.352772,-0.785884,1.747031,0.141522,...,0.070164,-0.142563,0.006056,0.151542,0.009402
4,-1.535240,-2.582608,-0.898018,0.405198,0.699272,...,-0.223594,0.124800,0.069084,0.042946,-0.058030
...,...,...,...,...,...,...,...,...,...,...,...
4995,2.744622,-3.227836,0.800372,0.581077,-0.293090,...,0.080491,0.043873,-0.039090,0.216866,-0.107188
4996,2.521653,5.929234,-0.288751,1.213837,3.842388,...,-0.231467,-0.088642,-0.108862,-0.071018,0.010272
4997,-5.671694,-0.559945,-1.204842,3.379102,-0.944850,...,-0.234581,-0.072824,-0.740899,-0.141401,-0.298439
4998,7.081308,1.454545,0.472432,1.113097,0.354064,...,-0.014282,0.192556,-0.009050,-0.175727,-0.156855


In [ ]:
# Helper function to export csv file storing predictions of a classifier on the blind test set 
def export_predictions(filename, classifier):
    predictions = []
    filepath = './Output/' + filename + '.csv'

    for i in output_test_norm:
        prediction = classifier.predict([list(i)])
        predictions.append(prediction[0])

    output_df = pd.DataFrame(predictions, columns = ['label'])
    output_df.to_csv(filepath, sep=",", float_format='%d', index_label="id")


In [ ]:
# export_predictions('knn_kaggle', knn)
export_predictions('svm_kaggle', svm)

Example Usage:

`export_predictions('knn', knn)`. This will create a file "knn.csv" in the Output folder which will store the predictions of the KNN classifier for the blind testing data. 